<a href="https://colab.research.google.com/github/JP-SystemsX/24cast/blob/master/AI_Grid_Hackathon_JP_DEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
!pip install -q wget
!pip install -q autogluon
!pip install -q pyarrow==15.0.2
!pip install -q pandas

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.0/428.0 kB 27.4 MB

In [2]:
!wget https://zenodo.org/records/3899018/files/LoadProfile_20IPs_2016.csv -q
!wget https://zenodo.org/records/3899018/files/LoadProfile_30IPs_2017.csv -q

In [3]:
!ls

LoadProfile_20IPs_2016.csv  LoadProfile_30IPs_2017.csv	sample_data


In [5]:
import pandas as pd
from copy import deepcopy
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from pathlib import Path

In [6]:
def train_test_split():
    lp_20ips_2016 = pd.read_csv("LoadProfile_20IPs_2016.csv", sep=";", skiprows=1)
    lp_30ips_2017 = pd.read_csv("LoadProfile_30IPs_2017.csv", sep=";", skiprows=1)
    Path("./tune").mkdir(exist_ok=True)
    Path("./test").mkdir(exist_ok=True)

    lp_20ips_2016["Time stamp"] = lp_20ips_2016["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
    lp_20ips_2016["Time stamp"] = pd.to_datetime(lp_20ips_2016.loc[:, "Time stamp"], format='%d.%m.%Y %H:%M:%S')

    lp_30ips_2017["Time stamp"] = lp_30ips_2017["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
    lp_30ips_2017["Time stamp"] = pd.to_datetime(lp_30ips_2017.loc[:, "Time stamp"], format='%d.%m.%Y %H:%M:%S')

    train_2016_full = lp_20ips_2016.iloc[:, :-7]
    train_2017_full = lp_30ips_2017.iloc[:, :-8]
    test_2016_full = pd.concat([lp_20ips_2016["Time stamp"], lp_20ips_2016.iloc[:, -7:] ], axis=1)
    test_2017_full = pd.concat([lp_30ips_2017["Time stamp"], lp_30ips_2017.iloc[:, -8:] ], axis=1)

    train_2016_training_data = train_2016_full[train_2016_full["Time stamp"].dt.month <= 8]
    train_2016_val_data = train_2016_full[train_2016_full["Time stamp"].dt.month > 8]

    test_2016_training_data = test_2016_full[test_2016_full["Time stamp"].dt.month <= 8]
    test_2016_test_data = test_2016_full[test_2016_full["Time stamp"].dt.month > 8]

    train_2017_training_data = train_2017_full[train_2017_full["Time stamp"].dt.month <= 8]
    train_2017_val_data = train_2017_full[train_2017_full["Time stamp"].dt.month > 8]

    test_2017_training_data = test_2017_full[test_2017_full["Time stamp"].dt.month <= 8]
    test_2017_test_data = test_2017_full[test_2017_full["Time stamp"].dt.month > 8]

    train_2016_training_data.to_csv("tune/2016_train.csv", index=False)
    train_2016_val_data.to_csv("tune/2016_val.csv", index=False)
    test_2016_training_data.to_csv("test/2016_train.csv", index=False)
    test_2016_test_data.to_csv("test/2016_test.csv", index=False)

    train_2017_training_data.to_csv("tune/2017_train.csv", index=False)
    train_2017_val_data.to_csv("tune/2017_val.csv", index=False)
    test_2017_training_data.to_csv("test/2017_train.csv", index=False)
    test_2017_test_data.to_csv("test/2017_test.csv", index=False)
train_test_split()

In [14]:
prediction_length = 48
long_runtime = (3*60-5)*60 # in seconds
dev_runtime = 5*60 # in seconds

train_1 = pd.read_csv("tune/2016_train.csv")
train_2 = pd.read_csv("tune/2017_train.csv")
val_1 = pd.read_csv("tune/2016_val.csv")
val_2 = pd.read_csv("tune/2017_val.csv")

def prepare_AG_data(dfs: list[pd.DataFrame]):
    # Prepare Data
    melted_dfs = []
    for df in dfs:
        df = deepcopy(df)#[["Time stamp", "LG 1"]]
        df.loc[:, ["Time stamp"]] = df["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
        df.loc[:, ["Time stamp"]] = pd.to_datetime(df["Time stamp"])
        # TODO Normalize Data else MASE means nothing
        # Concat Columns & Assign Ids
        df = df.melt(id_vars=['Time stamp'], var_name='item_id', value_name='target')
        df = df.dropna()
        df.interpolate(method='linear', inplace=True)
        melted_dfs.append(df)
    df = pd.concat(melted_dfs)
    df = TimeSeriesDataFrame.from_data_frame(
        df,
        id_column="item_id",
        timestamp_column="Time stamp"
    )
    return df

train = prepare_AG_data([train_1, train_2])
val = prepare_AG_data([val_1, val_2])
train

<ipython-input-14-45395cf153cc>:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear', inplace=True)
<ipython-input-14-45395cf153cc>:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear', inplace=True)
<ipython-input-14-45395cf153cc>:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear', inplace=True)
<ipython-input-14-45395cf153cc>:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear'

target
item_id timestamp                  
LG 1    2016-01-01 00:15:00    39.0
        2016-01-01 00:30:00    41.0
        2016-01-01 00:45:00    39.0
        2016-01-01 01:00:00    40.0
        2016-01-01 01:15:00    39.0
...                             ...
LG 22   2017-08-31 22:45:00     1.5
        2017-08-31 23:00:00     1.6
        2017-08-31 23:15:00     1.4
        2017-08-31 23:30:00     1.0
        2017-08-31 23:45:00     0.7

[817706 rows x 1 columns]

In [8]:
%%time

predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    path="autogluon-m4-hourly",
    target="target",
    eval_metric="MAPE",
    freq="15min"
)

# Dev Setting
predictor = predictor.fit(
    train,
    presets="medium_quality",
    time_limit=dev_runtime,
)

Frequency '15min' stored as '15T'
Beginning AutoGluon training... Time limit = 300s
AutoGluon will save models to 'autogluon-m4-hourly'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          1
Memory Avail:       11.10 GB / 12.67 GB (87.6%)
Disk Space Avail:   70.25 GB / 112.64 GB (62.4%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': '15T',
 'hyperparameters': 'light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 48,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'time_limit': 300,
 'verbosity': 2}

train_data with frequency 'No

CPU times: user 2min 10s, sys: 7.28 s, total: 2min 17s
Wall time: 2min 22s


In [9]:
predictor.evaluate(val)

data with frequency 'None' has been resampled to frequency '15T'.
Model not specified in predict, will default to the model with the best validation score: RecursiveTabular


{'MAPE': -0.5511686378094538}

In [10]:
predictor.leaderboard()

,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-2.269861,5.152582,1.266239,7
1,RecursiveTabular,-2.269861,5.152582,30.259968,3
2,Theta,-2.335450,23.646065,1.436210,6
3,ETS,-2.335450,32.021313,1.871572,5
4,DirectTabular,-4.192494,2.530302,30.999488,4
5,SeasonalNaive,-8.322310,0.245906,1.334401,2
6,Naive,-8.322310,0.206660,1.317565,1


# Level UP

In [ ]:
%%time

predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    path="autogluon-m4-hourly",
    target="target",
    eval_metric="MAPE",
    freq="15min"
)

# TODO Increase Time Limit For Final Version
predictor = predictor.fit(
    train,
    presets="best_quality",
    time_limit=long_runtime,
)

Frequency '15min' stored as '15T'
Beginning AutoGluon training... Time limit = 10500s
AutoGluon will save models to 'autogluon-m4-hourly'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          1
Memory Avail:       9.45 GB / 12.67 GB (74.6%)
Disk Space Avail:   69.98 GB / 112.64 GB (62.1%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': '15T',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 48,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'time_limit': 10500,
 'verbosity': 2}

train_data with frequency 

In [ ]:
print(predictor.evaluate(val))
predictor.leaderboard()

## Full Search

In [ ]:
# Model Exploration Exploration
# TODO Custom Config that compares all Models from Model Zoo
predictor = TimeSeriesPredictor(prediction_length=prediction_length)
predictor = predictor.fit(
    train,
    hyperparameters={
        "Chronos": {
            "model_path": "tiny",
            "batch_size": 64,
            "device": "cpu",
        },
        "NaiveModel": {},
        "SeasonalNaiveModel": {},
        "AverageModel": {},
        "SeasonalAverageModel": {},
        "ZeroModel": {},
        "ETSModel": {},
        "AutoARIMAModel": {},
        "AutoETSModel": {},
        "AutoCESModel": {},
        "ThetaModel": {},
        "ADIDAModel": {},
        "CrostonClassicModel": {},
        "CrostonOptimizedModel": {},
        "CrostonSBAModel": {},
        "IMAPAModel": {},
        "NPTSModel": {},
        "DeepARModel": {},
        "DLinearModel": {},
        "PatchTSTModel": {},
        "SimpleFeedForwardModel": {},
        "TemporalFusionTransformerModel": {},
        "TiDEModel": {},
        "WaveNetModel": {},
        "DirectTabularModel": {},
        "RecursiveTabularModel": {},
    },
    # skip running backtests during `fit`, as only consider 1 model
    skip_model_selection=True,
    verbosity=0,
)
# TODO Train for a few hours (with GPU)
predictor = predictor.fit(
    train,
    presets="best_quality",
    time_limit=long_runtime, # Colab Timeout
)

# TODO Investigate Leaderboard
print(predictor.evaluate(val))
predictor.leaderboard()

In [ ]:
# TODO Tune best found Config

# Chronos

In [ ]:
# TODO Test out Tiny Chronos compare to Large Chronos -> GPU and delay Worth it
# TODO Is Finetuned Tiny much better than untuned Tiny
# TODO Speed up Chronos
# TODO How compare to classical AG